In [127]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 21 09:29:09 2023

@author: maggie

Updated 5/14/2024 - Charlotte Stanley

script to summarize tabular results of treatments, severity

"""

import pandas as pd
import arcpy 
import os

In [131]:
# set two suffixes - to run code twice for results with and without cal fire data
suffix_list = ['noCF','CF']

In [135]:
endyear = 2023 


## functions to create reporting tables

In [142]:
#severity reporting

def create_severity_reporting_full_region():
    ''' This function reports the acres of each severity type.
    '''    
    for i in range(2000, endyear):
        print ('working on non fte severity reporting for ' + str(i))
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + '_' + str(i) +'.csv'))
        df2 = df.groupby('SEVERITY')['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, 'SEVERITY_SUMMARY_NO_FTE_' + str(i) +'.csv'), index=False)


def create_severity_reporting_with_FTE():
    '''this function reports the acres of each severity type for each FTE type.
    '''
    for i in range(2000, endyear):
        print(f'working on reporting severity with fte for {i}')
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + '_' + str(i) +'.csv'))
        df2 = df.groupby(['FTE', 'SEVERITY'])['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, f'SEVERITY_SUMMARY_FTE_{i}.csv'), index = False )

In [143]:
#Treatment reporting

def create_treatment_reporting_full_region():
    ''' This function reports the treatments that took place that year and how many acres were treated for each type. 
    '''
    for i in range(2000, endyear):
        print('working on reporting treatment for' + str(i))
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + f'_{i}.csv'))
        df2 = df.groupby('Treatment')['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, f'Treatment_summary_{i}.csv'), index = False)
        

def create_treatment_reporting_by_severity():
    ''' This function reports the treatment types and how many acres were treated for each severity class. 
    '''
    for i in range(2000, endyear):
        print(f'working on reporting treatment by sev for {i}')
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + f'_{i}.csv'))
        df2 = df.groupby(['SEVERITY', 'Treatment'])['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, f'Treatment_summary_severity_{i}.csv'), index = False)
        
def create_treatment_reporting_by_FTE():
    ''' This function reports the treatment types and acres treated for each FTE type.
    '''
    for i in range(2000, endyear):
        print(f'working on reporting treatment by FTE for {i}')
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + f'_{i}.csv'))
        df2 = df.groupby(['FTE', 'Treatment'])['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, f'Treatment_summary_FTE_{i}.csv'), index = False)

def create_treatment_reporting_FTE_sev():
    ''' This function reports the treatment types and treated acreage for each severity class within each FTE
    '''
    for i in range(2000, endyear):
        print(f'working on reporting treatment for severity and by FTE for {i}')
        df = pd.read_csv(os.path.join(tables_folder, 'values_' + name + f'_{i}.csv'))
        df2 = df.groupby(['FTE', 'SEVERITY', 'Treatment'])['Acres'].sum()
        df2 = df2.reset_index()
        df2.to_csv(os.path.join(tables_folder, f'Treatment_Summary_FTE_severity_{i}.csv'), index= False) #without descriptions


In [144]:
def create_sums_allyears_reporting():
    ''' This function sums across all years of acres of treatment for each severity type for each FTE using the tables from
    the create_treatment_reporting_FTE_sev function. 
    '''
    
    #read csvs into dataframes for Treatment_Summary_FTE_severity_YEAR 
    all_dfs = []
    for i in range (2000, endyear):
        print('working in reporting treatment and severity by FTE for all years')
        df = pd.read_csv(os.path.join(tables_folder, f'Treatment_Summary_FTE_severity_{i}.csv'))
        all_dfs.append(df)
        
    #merge and sum duplicates
    cf = pd.concat(all_dfs, axis = 0)
    print('concatenated all dataframes, keeping column names')
    cf= cf.groupby(['FTE', 'SEVERITY', 'Treatment',])['Acres'].sum()
    cf = cf.reset_index()
    print('exporting summary csv')
    cf.to_csv(os.path.join(tables_folder, 'FTE_Treatment_Sev_allyears.csv'), index = False)

In [145]:
def create_csv_allyears_treatment():
    '''
    This function will result in a csv that list acres of treatment for each FTE AND year. 
    Uses the Treatment_summary_FTE_YEAR csvs.
    '''
    list_dfs = []
    for i in range(2000, endyear):
        print(f'adding df for year {i}')
        df = pd.read_csv(os.path.join(tables_folder, f'Treatment_summary_FTE_{i}.csv'))
        df.insert(0, 'Year', i)
        df = df[['Year', 'FTE', 'Treatment', 'Acres']]
        list_dfs.append(df)
    
    cf = pd.concat(list_dfs, axis =0)
    print('concatenated all dataframes, keeping column names')
    cf = cf[['Year', 'FTE', 'Treatment', 'Acres']]
    # cf = cf.reset_index()
    print('exporting summary csv')
    cf.to_csv(os.path.join(tables_folder, 'FTE_Treatment_allyears.csv'), index = False)

In [146]:
def create_csv_allyears_severity():
    '''
    This function will result in a csv that lists acres of severity for each FTE AND year.
    Uses the SEVERITY_SUMMARY_FTE_Year csvs.
    '''
    list_dfs = []
    for i in range(2000, endyear):
        print(f'adding df for year {i}')
        df = pd.read_csv(os.path.join(tables_folder, f'SEVERITY_SUMMARY_FTE_{i}.csv'))
        df.insert(0, 'Year', i)
        df = df.rename(columns = {'SEVERITY': 'Severity Category'})
        df = df[['Year', 'FTE', 'Severity Category', 'Acres']]
        list_dfs.append(df)
    
    cf = pd.concat(list_dfs, axis =0)
    print('concatenated all dataframes, keeping column names')
    cf = cf[['Year', 'FTE', 'Severity Category', 'Acres']]
    # cf = cf.reset_index()
    print('exporting summary csv')
    cf.to_csv(os.path.join(tables_folder, 'FTE_Severity_allyears.csv'), index = False)

## run functions

In [147]:
# summarize results for two datasets

for name in suffix_list:
    # set folder location
    tables_folder = r"N:\Sierra_Nevada\Projects\State of the Sierra\State of the Sierra\TNC_tables_polygon\\" + name

    # severity summaries
    create_severity_reporting_with_FTE() #Burn severity by class AND FTE
    
    # treatment summaries
    create_treatment_reporting_full_region() #treatment full region
    create_treatment_reporting_by_severity() #Treatment by class (only done on areas that hadn't burned at high severity previously)
    create_treatment_reporting_by_FTE()  #Treatment by FTE
    create_treatment_reporting_FTE_sev() #Treatment by class AND FTE
    
    # all year summaries
    create_sums_allyears_reporting()
    create_csv_allyears_treatment()
    create_csv_allyears_severity()


working on reporting severity with fte for 2000
working on reporting severity with fte for 2001
working on reporting severity with fte for 2002
working on reporting severity with fte for 2003
working on reporting severity with fte for 2004
working on reporting severity with fte for 2005
working on reporting severity with fte for 2006
working on reporting severity with fte for 2007
working on reporting severity with fte for 2008
working on reporting severity with fte for 2009
working on reporting severity with fte for 2010
working on reporting severity with fte for 2011
working on reporting severity with fte for 2012
working on reporting severity with fte for 2013
working on reporting severity with fte for 2014
working on reporting severity with fte for 2015
working on reporting severity with fte for 2016
working on reporting severity with fte for 2017
working on reporting severity with fte for 2018
working on reporting severity with fte for 2019
working on reporting severity with fte f

adding df for year 2004
adding df for year 2005
adding df for year 2006
adding df for year 2007
adding df for year 2008
adding df for year 2009
adding df for year 2010
adding df for year 2011
adding df for year 2012
adding df for year 2013
adding df for year 2014
adding df for year 2015
adding df for year 2016
adding df for year 2017
adding df for year 2018
adding df for year 2019
adding df for year 2020
adding df for year 2021
adding df for year 2022
concatenated all dataframes, keeping column names
exporting summary csv
